In [4]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    #print(questions[:5])
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('answers_squad_format.json')
#val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [5]:
from transformers import pipeline

pipe = pipeline("question-answering", model="twmkn9/distilbert-base-uncased-squad2")

c:\Users\NIKITA_PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_answers[:2]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects c

In [7]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
#add_end_idx(val_answers, val_contexts)

In [8]:
train_answers[:5]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0,
  'answer_end': 779},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which 

In [9]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("twmkn9/distilbert-base-uncased-squad2")


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
#val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [10]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [11]:
len(train_encodings['input_ids'])

45

In [9]:
train_encodings['input_ids'][0]

[101,
 2108,
 2583,
 2000,
 6509,
 6304,
 19512,
 1010,
 2326,
 20202,
 2071,
 2490,
 2037,
 6304,
 2012,
 2151,
 2051,
 1998,
 2013,
 5973,
 1010,
 2302,
 9352,
 2383,
 2000,
 3942,
 1996,
 8013,
 2609,
 1012,
 6556,
 3229,
 2071,
 2036,
 2393,
 2000,
 2488,
 7374,
 2326,
 20202,
 2007,
 1996,
 4072,
 2592,
 2005,
 2037,
 8518,
 1012,
 4209,
 2055,
 1996,
 3698,
 2030,
 2291,
 7561,
 1999,
 2062,
 6987,
 25828,
 13169,
 7070,
 2051,
 2138,
 3223,
 8622,
 3033,
 2030,
 2060,
 3941,
 2064,
 3641,
 1010,
 4810,
 1998,
 2716,
 2247,
 1012,
 9308,
 1010,
 2349,
 2000,
 1037,
 7820,
 4668,
 2051,
 1010,
 1996,
 6304,
 6681,
 2024,
 2039,
 1998,
 2770,
 2172,
 5514,
 1010,
 7494,
 2068,
 2013,
 17047,
 2091,
 7292,
 1012,
 2005,
 1037,
 3698,
 1998,
 3941,
 7751,
 1010,
 2478,
 6556,
 3229,
 2036,
 2965,
 2008,
 1996,
 2168,
 2193,
 1997,
 2326,
 20202,
 2071,
 2490,
 2062,
 6304,
 2030,
 3749,
 3176,
 2578,
 1012,
 1037,
 5851,
 6556,
 4434,
 2000,
 5500,
 10394,
 1998,
 3941,
 2003,
 2036,

In [12]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. remote access could also help to better prepare service technicians with the necessary information for their tasks. knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. moreover, due to a shorter reaction time, the customers machines are up and running much faster, saving them from costly downtime. for a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services. a secure remote connection to distributed machinery and equipment is also the basis for many new concepts and services within industry 4. 0 such as predictive maintenance, where a secure connection is established to 

In [13]:
train_encodings.char_to_token(0, train_answers[0]['answer_start'])

1

In [14]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        go_back=1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - go_back)
            go_back+=1
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
#add_token_positions(val_encodings, val_answers)

In [15]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [16]:
train_encodings['start_positions'][:10]

[1, 1, 1, 148, 512, 1, 512, 323, 1, 512]

In [17]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
#val_dataset = SquadDataset(val_encodings)

In [18]:
print(type(train_dataset))

<class '__main__.SquadDataset'>


In [19]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("twmkn9/distilbert-base-uncased-squad2")

In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
model_path = 'model/distilbert-custom'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

In [30]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

In [31]:


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    loop=tqdm(train_loader)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

model.eval()

Epoch 2: 100%|██████████| 3/3 [11:24<00:00, 228.21s/it, loss=0.99]


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [32]:
model_path='model/distilbert-custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('model/distilbert-custom\\tokenizer_config.json',
 'model/distilbert-custom\\special_tokens_map.json',
 'model/distilbert-custom\\vocab.txt',
 'model/distilbert-custom\\added_tokens.json',
 'model/distilbert-custom\\tokenizer.json')

In [21]:
def get_answers(question, context):
    inputs = tokenizer.encode_plus(question, context, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)

    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))    
    return answer

In [22]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is IIT?"
inputs = tokenizer.encode_plus(question, context, truncation=True, padding=True, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
print(tokenizer.decode(input_ids[0]))
print(attention_mask[0])
# with torch.no_grad():
#     output = model(input_ids, attention_mask=attention_mask)

# answer_start = torch.argmax(output.start_logits)
# print(answer_start)
# answer_end = torch.argmax(output.end_logits)
# print(answer_end)


# answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))    
# print(answer)

[CLS] what is iit? [SEP] the foundation of smart manufacturing at the indian institute of technology ( iit ) is a research and development center dedicated to advancing the field of smart manufacturing in india. iits are a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology. the foundation of smart manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems. its primary goal is to develop and implement cutting - edge technologies to enhance the productivity, quality, and sustainability of indian manufacturing industries. during the months of june - july 2021, fsm organized a summer online internship. 900 + applications were received and 150 + students were accepted for internships in various domains such as augmented reality, machine learning, automation, iiot, realtime dashboards, and robotics. more than 45

In [23]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is nmghkjkj?"
print(f"Predicted Answer: {get_answers(question, context)}")
#input_ids=test_encodings['input_ids'].to(device)
#outputs=model(test_encodings['input_ids'],test_encodings['attention_mask'])

Predicted Answer: the foundation of smart manufacturing at the indian institute of technology ( iit ) is a research and development center dedicated to advancing the field of smart manufacturing in india. iits are a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology


In [24]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is IIT?"
print(f"Predicted Answer: {get_answers(question, context)}")
#input_ids=test_encodings['input_ids'].to(device)
#outputs=model(test_encodings['input_ids'],test_encodings['attention_mask'])

Predicted Answer: a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology


Answer of above question

Predicted Answer: a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology

In [25]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="What is Foundation of Smart Manufacturing?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware. fsm is focussed on enabling people from becoming smart to samarth ( capable ) fraternity by creating awareness, step - wise learning and becoming an expert from domains driving industry 4. 0. academic strength of iit delhi brought interdisciplinary learnings to fsm. expert faculty members from various departments contribute to it. fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and ind

Answer of above que

Predicted Answer: foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware. fsm is focussed on enabling people from becoming smart to samarth ( capable ) fraternity by creating awareness, step - wise learning and becoming an expert from domains driving industry 4. 0. academic strength of iit delhi brought interdisciplinary learnings to fsm. expert faculty members from various departments contribute to it. fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and industry partners. m. tech and ph. d students doing research in smart manufacturing are contributing to fsm technologies and increase its knowledgebase. fsm brought industry partners from mncs operating all across the globe to bring rich experience in the smart manufacturing technologies. automation industries association ( aia ) in its quest to set up a common engineering facility center ( cefc ) at iit delhi has brought together experts from various industries in the sector of automation. these experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in india. the following firms have agreed to come together to set up the cefc. they comprise technology & investment partners, simulation & amp ; integration partners and machinery partners


In [26]:
context=" The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="Explain internship?"
print(f"Predicted Answer: {get_answers(question, context)}")


Predicted Answer: the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months.


Predicted Answer: the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months.

In [27]:

context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="What are contact information ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: contact us today for implementing. call us for any query011 - 26582053, 807


In [28]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What on December 22, 2022?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune


Answr of above que

Predicted Answer: we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune

In [29]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What is full-form DEWG?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: iitd - aia foundation for smart manufacturing is delighted to announce the successful conclusion of our week long training program titled ' competency building in industry 4. 0 ' for industry experts from hindustan aeronautics limited ( hal ) in the field of smartmanufacturing. hal has entrusted us with the task of upskilling their experts from various domains. our carefully crafted learning sessions provided valuable insights into the major domains of industry 4. 0. including automation, iiot, augmentedreality, robotics, ai, mliitd - aia foundation for smart manufacturing got an opportunity to be a part of g20 digital economy working group ( dewg ) in lucknow hosted by up government.


Answer of above question:
Predicted Answer: mliitd - aia foundation for smart manufacturing got an opportunity to be a part of g20 digital economy working group ( dewg ) in lucknow hosted by up government.

In [30]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What is HAL?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: hal has entrusted us with the task of upskilling their experts from various domains. our carefully crafted learning sessions provided valuable insights into the major domains of industry 4. 0. including automation, iiot, augmentedreality, robotics, ai, mliitd - aia foundation for smart manufacturing got an opportunity to be a part of g20 digital economy working group ( dewg ) in lucknow hosted by up government. these type of events helps all the segments of nation like politicians, economic dignitaries, business leader, entrepreneurs, industry associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.


In [31]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions."
question="What are the service provided?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions


Predicted Answer: the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions

In [32]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation Integration Partners and Machinery Partners."
question="Explain partners?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: contact us today for implementing call us for any query011 - 26582053, 8076197190 or email us oninfo @ iafsm. in. the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months. the focus of the internship is on the theme of ' smart factory, ' which is a rapidly growing industry. this internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. it is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. if you are interested, don ' t miss this chance to be a part of the program and enhance your career prospects.. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubat

In [33]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware.. Fill out the online registration form Shortlisting based on the eligibility criteria Level 1: Learning modules (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023. Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship.    Level 2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023. Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors. Final Selection and Project Allocation based on the Level 2 performance and preferences These are full-time internships and require 48 hours per week    "
question="what are the process for internship?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: this internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. it is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. if you are interested, don ' t miss this chance to be a part of the program and enhance your career prospects.. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware.. fill out the online registration form shortlisting based on the eligibility criteria level 

In [34]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is FSM Masterclass Series?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: a 4 - month program, having 4 different masterclasses for technologies such as “ augmented reality ”, “ machine learning ”, “ automation ”, and “ iiot ”


In [35]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews.Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available.Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology.Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are 'enabling technologies' which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a “thing” in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized."

question="What is Cyber Physical System?"
print(f"Predicted Answer: {get_answers(question, context)}")




Predicted Answer: a cyber - physical system ( cps ) is a “ thing ” in the internet of things


In [36]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews.Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available.Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology.Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are 'enabling technologies' which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a “thing” in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized."

question="Explain Cyber Physical System?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: cyber - physical systems provide the basis the creation of an internet of things ( iot ) which makes smart services and products possible


In [37]:
question="What is Modules/Trainings covered in the kit?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: the fsm masterclass series was a 4 - month program, having 4 different masterclasses for technologies such as “ augmented reality ”, “ machine learning ”, “ automation ”, and “ iiot ”. the program was attended by college students, college professors and employees from various industries. each course was a month - long program, consisting of 4 live classes, 4 hands - on lab sessions, e - learning modules and 1 major project


In [38]:
context="Additionally it demonstrates the HMI Designing and Dashboard designing feature. This is the third kit of the smart trainer modules used to demonstrate a working manufacturing system. It has a servo motor and drive to rotate an indexing table. Along with the motion feature, it has four stations which can be used to demonstrate multiple operations. Each station has smart inductive sensors to sense the part  ‘s presence which consists of the distance and proximity sensors, servo drive, test product, RFID Reader, Solenoid, USB Camera mounted on them.Station 1: It has a RFID controller used to read and write the tag values of a product. It communicates with IO Link master (present in PLC trainer kit) and IO link master communicates the data with the PLC. So, RFID operation and use case is demonstrated. Station 2: This station consists of a distance sensor used to calculate the height or distance of the product. It is a smart sensor and communicates with IO Link master. So, along with the analog value it communicates it  2019s status to the PLC as well. Verification of product dimension is also demonstrated. Station 3: This station has a vision camera installed on it. This camera is used for image processing and product testing. Sorting operation is performed using the image processing feature. Station 4: This station has a solenoid which is directly connected with the digital output of the PLC. Extension and retraction of the solenoid is programmed in the PLC for this operation.This mechanism kit is used to demonstrate and train motion configuration and programming, Vision System, Smart Sensing, Machine Learning, Augmented Reality.  Programming Kit HMI Kit Mechanism Kit It is showing the working of Smart Manufacturing Kit Education & Training Device Services Research & Skill Certification  Software-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell demonstrates a flexible manufacturing cell. This cell is an assembly line where components of a main part gets assembled, the main part comprises of minimum of 7 sub-parts and more, which are then assembled by placing, inserting and part-screwing the main part. It has multiple stations which performs operation on the parts (like valve body assembly, vision inspection, testing, packaging). The main feature of the cell is exhibited using the programming, control and diagnosis of the robotic manipulator amongst other peripheral and safety devices. We call it as Multi Process robotic cell because it can be used for more than one manufacturing process.  There are different stations in the robotic cell: 1. Conveyor Station: Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station: All the parts (axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done.3.Testing Station: All the pneumatic ports of the valve body will be tested in this station. If there is any leakage,"
question="What is Staion1?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: station 2 : this station consists of a distance sensor used to calculate the height or distance of the product. it is a smart sensor and communicates with io link master


In [39]:
question="What is Modules/Trainings covered in the kit?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: 


In [40]:
question="What is Modules/Trainings covered in the kit?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: 


In [41]:
question="Explain Station 1?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: station 1 : it has a rfid controller used to read and write the tag values of a product. it communicates with io link master ( present in plc trainer kit ) and io link master communicates the data with the plc. so, rfid operation and use case is demonstrated. station 2 : this station consists of a distance sensor used to calculate the height or distance of the product. it is a smart sensor and communicates with io link master. so, along with the analog value it communicates it 2019s status to the plc as well. verification of product dimension is also demonstrated. station 3 : this station has a vision camera installed on it. this camera is used for image processing and product testing. sorting operation is performed using the image processing feature. station 4 : this station has a solenoid which is directly connected with the digital output of the plc. extension and retraction of the solenoid is programmed in the plc for this operation. this mechanism kit is used to 

In [42]:
question="what is Station 2?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: this station consists of a distance sensor used to calculate the height or distance of the product. it is a smart sensor and communicates with io link master


In [43]:

question="why  it is called as  Multi Process robotic cell ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: in multi process robotic cell demonstrates a flexible manufacturing cell


In [44]:
context="At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory . "
question="what is CEFC ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: fsm brought industry partners from mncs operating all across the globe to bring rich experience in the smart manufacturing technologies. automation industries association ( aia ) in its quest to set up a common engineering facility center


In [45]:
context="At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory . "
question="what is date for internship ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: at fsm we provide various client services and training under the domain of smart manufacturing. the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware. fsm is focussed on enabling people from becoming smart to samarth ( capable ) fraternity by creating awareness, step - wise learning and becoming an ex

In [46]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("twmkn9/distilbert-base-uncased-squad2")


In [47]:
tokens=tokenizer.encode_plus(context,add_special_tokens=True,max_length=512,truncation=True,padding="max_length")
tokens

{'input_ids': [101, 2012, 1042, 6491, 2057, 3073, 2536, 7396, 2578, 1998, 2731, 2104, 1996, 5884, 1997, 6047, 5814, 1012, 1996, 2578, 2097, 9585, 7846, 2000, 3231, 1998, 2139, 8569, 2290, 3471, 2077, 23658, 1996, 5576, 2012, 1996, 2609, 1004, 2036, 3499, 2068, 2000, 7551, 1998, 7601, 7103, 2618, 2007, 2019, 6413, 4666, 1997, 3115, 1998, 28749, 7300, 1012, 3192, 2005, 6047, 5814, 1006, 1042, 6491, 1007, 7126, 1010, 6753, 1010, 1998, 11791, 6047, 5814, 8474, 2005, 2796, 3068, 2000, 7409, 1010, 2801, 2618, 1010, 1998, 3046, 2041, 1999, 2037, 4264, 1012, 1042, 6491, 2036, 8704, 2005, 1037, 7570, 27348, 4547, 8882, 1998, 8066, 1011, 2311, 2565, 2083, 1037, 17026, 4297, 19761, 3508, 1998, 3831, 4044, 1012, 1042, 6491, 4813, 2003, 1037, 7570, 27348, 4132, 2005, 4346, 10047, 16862, 3512, 2731, 3325, 2083, 2444, 8921, 1010, 3784, 4083, 1010, 2444, 13616, 1010, 2444, 13625, 1004, 2969, 13823, 11110, 2006, 19512, 7801, 5025, 8051, 1012, 1042, 6491, 2003, 3579, 6924, 2006, 12067, 2111, 2013, 3352,

In [48]:
tokens=tokenizer.encode_plus(context,add_special_tokens=False,return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors


In [49]:
tokens

{'input_ids': tensor([[ 2012,  1042,  6491,  2057,  3073,  2536,  7396,  2578,  1998,  2731,
          2104,  1996,  5884,  1997,  6047,  5814,  1012,  1996,  2578,  2097,
          9585,  7846,  2000,  3231,  1998,  2139,  8569,  2290,  3471,  2077,
         23658,  1996,  5576,  2012,  1996,  2609,  1004,  2036,  3499,  2068,
          2000,  7551,  1998,  7601,  7103,  2618,  2007,  2019,  6413,  4666,
          1997,  3115,  1998, 28749,  7300,  1012,  3192,  2005,  6047,  5814,
          1006,  1042,  6491,  1007,  7126,  1010,  6753,  1010,  1998, 11791,
          6047,  5814,  8474,  2005,  2796,  3068,  2000,  7409,  1010,  2801,
          2618,  1010,  1998,  3046,  2041,  1999,  2037,  4264,  1012,  1042,
          6491,  2036,  8704,  2005,  1037,  7570, 27348,  4547,  8882,  1998,
          8066,  1011,  2311,  2565,  2083,  1037, 17026,  4297, 19761,  3508,
          1998,  3831,  4044,  1012,  1042,  6491,  4813,  2003,  1037,  7570,
         27348,  4132,  2005,  4346, 1

In [50]:
len(tokens['input_ids'][0])

598

In [51]:
input_ids_chunks=tokens['input_ids'][0].split(510)
mask_chunks=tokens['attention_mask'][0].split(510)

for tensor in input_ids_chunks:
    print(len(tensor))
   

510
88


In [52]:
import torch

chunksize=512
input_ids_chunks= list(input_ids_chunks)
mask_chunks= list(mask_chunks)
print(len(input_ids_chunks))
for i in range(len(input_ids_chunks)):
    input_ids_chunks[i]=torch.cat([torch.Tensor([101]),input_ids_chunks[i],torch.Tensor([102])])
    mask_chunks[i]=torch.cat([torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])])
    
    pad_len=chunksize-input_ids_chunks[i].shape[0]
    if pad_len>0:
        input_ids_chunks[i]=torch.cat([input_ids_chunks[i],torch.Tensor([0]* pad_len)])
        mask_chunks[i]=torch.cat([mask_chunks[i],torch.Tensor([0]* pad_len)])

for chunk in input_ids_chunks:
    print(len(chunk))

2
512
512


In [53]:
input_ids=torch.stack(input_ids_chunks)
attention_mask=torch.stack(mask_chunks)

input_dict={
    "input_ids":input_ids.long(),
    "attention_mask":attention_mask.int()
}
print(len(input_dict["input_ids"]))
input_dict

2


{'input_ids': tensor([[ 101, 2012, 1042,  ..., 2219, 4264,  102],
         [ 101, 1012, 1996,  ...,    0,    0,    0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [54]:
import torch
def context_conversion(input):
    
    tokens=tokenizer.encode_plus(input,add_special_tokens=False,return_tensors="pt")
    input_ids_chunks=tokens['input_ids'][0].split(509)
    mask_chunks=tokens['attention_mask'][0].split(509)
    
    input_ids_chunks= list(input_ids_chunks)
    mask_chunks= list(mask_chunks)
    
    tensor1=input_ids_chunks[0]
    tensor2=torch.Tensor([102])
    given_word=torch.Tensor([1029])
    index = (tensor1 == given_word).nonzero()
    index = index[0]
    tensor1_part1 = tensor1[:index+1]
    tensor1_part2 = tensor1[index+1:]
    input_ids_chunks[0] = torch.cat((tensor1_part1, tensor2, tensor1_part2), dim=0)

    
    tensor_mask1=mask_chunks[0]
    tensor_mask2=torch.Tensor([1])
    index = index[0]
    tensor1_mask_part1 = tensor_mask1[:index+1]
    tensor1_mask_part2 = tensor_mask1[index+1:]
    mask_chunks[0] = torch.cat((tensor1_mask_part1, tensor_mask2, tensor1_mask_part2), dim=0)

   

    chunksize=512
    # input_ids_chunks= list(input_ids_chunks)
    # mask_chunks= list(mask_chunks)
    
    
    for i in range(len(input_ids_chunks)):
        input_ids_chunks[i]=torch.cat([torch.Tensor([101]),input_ids_chunks[i],torch.Tensor([102])])
        mask_chunks[i]=torch.cat([torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])])
        
        pad_len=chunksize-input_ids_chunks[i].shape[0]
        if pad_len>0:
            input_ids_chunks[i]=torch.cat([input_ids_chunks[i],torch.Tensor([0]* pad_len)])
            mask_chunks[i]=torch.cat([mask_chunks[i],torch.Tensor([0]* pad_len)])
    

    input_ids=torch.stack(input_ids_chunks)
    attention_mask=torch.stack(mask_chunks)
    
    input_dict={
        "input_ids":input_ids.long(),
        "attention_mask":attention_mask.int()
    }
    return input_dict

In [2]:
context=" At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. ,   Consulting ,  Skill Certification ,  Research ,  Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment.FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. ,  Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. ,  FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. ,  M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. ,  FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. ,  The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. ,    Fill out the online registration form Shortlisting based on the eligibility criteria Level 1: Learning modules (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023. Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship.   Level 2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023.  Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors.  Final Selection and Project Allocation based on the Level 2 performance and preferences These are full-time internships and require 48 hours per week  ,  The robot becomes the third arm of the human operator. This new form of collaboration opens up previously inconceivable possibilities for the smart factory of the future. As collaborative robots operate without physical safeguards, they have to permanently calculate the risk of colliding with humans, constantly checking this via the robot controller.   Collaborative robots works hand in hand with the operator, thereby enabling him to work more efficiently, more ergonomically, more precisely and with greater concentration. As a robot that can genuinely be deployed universally, it is defining new standards on the road to the fourth industrial revolution. Collaborative robots can be programmed onsite by the employees and can be quickly re-tasked to do multiple jobs quickly. They are lightweight and can be moved around easily in the production facility.   Collaborative robot have integrated sensors, passive compliance or over current detection as safety features. The integrated sensors will feel external forces and, if this force is too high, the robot will stop its movement. Passive compliance is produced by mechanical components. If an external force acts on a joint, this joint will submit itself to this force. So, in case of a collision, the joint will move in the opposite direction avoiding any injury. Also, an over current can be detected when a collision occurs. This is another safety feature, because the software can generate a security stop when it detects a current spike. Some of the applications of collaborative robots can be:   Assembly  Soldering Vision based quality inspection Machine Tending Gluing Screwing Pick and place Operating hand tools Laboratory work   ,  Multi Process Robotic Cell demonstrates a flexible manufacturing cell. This cell is an assembly line where components of a main part gets assembled, the main part comprises of minimum of 7 sub-parts and more, which are then assembled by placing, inserting and part-screwing the main part. It has multiple stations which performs operation on the parts (like valve body assembly, vision inspection, testing, packaging). The main feature of the cell is exhibited using the programming, control and diagnosis of the robotic manipulator amongst other peripheral and safety devices. We call it as Multi Process robotic cell because it can be used for more than one manufacturing process.  There are different stations in the robotic cell: 1. Conveyor Station: Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station: All the parts (axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done.3.Testing Station: All the pneumatic ports of the valve body will be tested in this station. If there is any leakage, valve body will be rejected.4. Hex-nut Assembly Station:- Hex nut will be assembled on the body in this station.5. Packaging Station: Assembled valve bodies will be placed in a box and packaging is done. A printing head will print the required details( type of bodies, number of bodies, date of assembly etc)  on the box and then the box will be dispatched. ,  The video here demonstrates the standalone function of loading inventory using trays from conveyor station to assembly station. The tray has multiple arrays of different sized part but same variety. There are trays with 3/2, 5/2, 5/2 double solenoid, 5/3 solenoid valve body. The end-effector of the robot have more than one tool attachments. This is possible due to tool changer allowing robot to perform multiple actions like pick and place, part screwing,  gripping via vacuum suction cup, bar code part scanning. ,  Training ,  <-Cyber Physical assembly line ,  Robotic Welding Cell-> ,  This Multi-purpose Robotic Cell is assembling different variants of direction control valves. It has different stations which performs different operations on the valve body. Assembly,Vision Inspection,Testing,Packaging are some of the operations that can be performed by this cell. The main feature of this cell is that all the programming is done on PLC, use of teach pendant or robot controller programming has been Bypassed. As we can use this cell for multiple purpose,we call it as Multi-Process Robotic Cell. ,  There are different station in the robotic cell:1.Conveyor station:- Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station:- All the parts (Axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done in this station.3.Testing Station:- All the ports of the valve body will be tested in this station. If there is any leakage, valve body will be rejected.4.Hex nut assembly station:- Hex nut will be placed on the body in this station.5. Packaging station :- Assembled valve bodies will be placed in a box and packaging is done. A printing head will print the required details( Type of bodies ,number of bodies, date of assembly etc\u2026) on the box and then the box will be dispatched. ,  1.Siemens PLC:- Siemens S7 1516 is the master plc in this station and it is responsible for all the programming in the cell.2. DI and DO: - 32-point digital input and 32 point digital output module is connected on the base rack with the plc.3. 6 Axis Kuka robot and one axis:- 6 Axis kuka robot performs all the motion operation in the cell.7th axis rotates the valve body during assembly and inspection.4. QR code scanner:- QR code scanner performs the inventory check as it scans the QR code present on body. After scanning a body, plc gets the data of that particular body and its location on the tray.5. Laser based Vision sensor:- This sensor inspects all the four faces of the body and if it is OK then the station will pass the body to next station or else it will reject the body and picks another body of the same type.6. Tool Changers :- Different slave tool changersare used for different operations. These slave tool changers engages with the master tool changer(Present on the Robot arm) when required. Tool changers in this cell are used for:- QR code scanner,Pneumatic gripper and Screwdriver, Vacuum Gripper ,  Software ,  Mobile Collaborative Robot  ,  Cyber Physical Factory ,  We are providing following services:  3D printing Smart Manufacturing Training Kits for Industrial Automation learning MIG Welding (including Wire-arc-additive-manufacturing)  ,  Multimaterial 3D printer ,  Smart Manufacturing Training Kits ,  MIG Welding Cell ,  A system consisting of 3 parts 1. Python code 2. Notification System 3. AR Application The system is designed to automatically detect any error occurring in the Work Station 1 and raising an alert, the user then using HoloLens 2 will follow the steps to resolve the error ,  IIoTAugmented Reality ,  The Multi-material Additive Manufacturing Machine is a versatile machine made by Hyrel 3D and AMS. It is not 'just a 3D printer'. It is a Large format, Rapid Prototyping, Research, Manufacturing and Educational Platform. This 3D printer is customizable on various levels: build volume, print bed, extruder, and more. It is equipped with the capacity to attach 5 tool heads simulatneously. We have tool heads for printing common FDM filaments(PLA, ABS, PC, PEEK), clays, biologicals, flexibles, UV-curables, RTV Silicones, etc. It is highly evolutive and also offers options for 4th and 5th axes, PCB milling tool, 3-phase machining tool and CO2 or Diode lasers (laser cutting). ,  Smart Manufacturing Training Kit-> ,  Cyber Physical Assembly Line demonstrates smart manufacturing and comes with different functionalities. It includes the Custom order processing, valve body, spool, solenoid, connector, valve functioning testing, packaging, inkjet printing. It consists of three work stations and one MPRC.Workstation 1: It is also known as Raw part storage, part loading and quality check. It uses the Rockwell PLC. It stores Product and do Pick & place arrangement for mounting the product on pallet.  Workstation 2: It is also known as axisymmetric part storage quality check and inspection work station. It uses B & R PLC. It stores spool and do pick and place arrangement for inserting the spool in valve body. It uses proximity sensor for inventory check. Workstation 3: It is also known as Prismatic Part Assembly and Screwing Workstation. It uses the Mitsubishi PLC. It stores all prismatic parts. It is used to place and tighten all the parts to valve body.Multi Process Robotic Cell (MPRC) :Click here to know more. Conclusion: All the station can run in Auto/Manual Mode. This is how the cyber physical assembly line works. There are features in the line such as smart sensing, Machine to Machine communication, smart dashboard, remote monitoring, alerts and notifications on email, safety, remote maintenance and Augmented Reality. ,  Simulation & Testing Services ,  Prototyping, Consulting Services ,  Skills Certification ,  Site Integration Services ,  Job Work & Research ,  Multi Process Robotic Cell-> ,  Augmented reality is one of the cutting edge technologies involved in the industry 4.0 trend when talking about smart manufacturing, it is a technology which was seen just as a fancy toy until a few years back, but which has now reached the right level of maturity to be employed in a productive environment.    AR applications can become even more powerful by augmenting them with data generated by another set of technologies, revolutionizing the marketplace called the Internet of Things (IoT). Combining AR and the IoT allows employees, such as field maintenance workers, to access vital machine operating statistics to assist in maintenance decisions, or an assembly worker to visually receive pick and place instructions and alerts if tools are working incorrectly or are out of tolerance.   Key AR ecosystem components include device hardware, an OS open to third-party application development, and a platform for software, as well as content creation and management. The standard device hardware for viewing AR applications has predominantly been smartphones and tablets due to their wide availability and exceptional compute power. The newer devices expected to accelerate enterprise AR adoption because they allow hands-free access to information are smart glasses, HMDs, and other digital eyewear solutions from companies such as Microsoft, ODG, Epson, and Vuzix.   Augmented reality renders 3D objects in real time into your field of vision. For example, looking through the cover of a machine and seeing its internal component structure.   Augmented Reality (AR) is a technology enriching the real world with digital information and media, such as 3D models and videos, overlaying in real-time the camera view of your smartphone, tablet, PC or connected glasses.   On the spectrum between virtual reality, which creates immersive, computer-generated environments, and the real world, augmented reality is closer to the real world. Augmented reality adds graphics, sounds, haptic feedback and smell to the natural world as it exists. Both video games and cell phones are driving the development of augmented reality.   The methods by which AR is achieved can generally be split into two broad categories: Marker-based, and markerless.   Marker-based Systems: This technology uses physical-world symbols as a reference point for computer graphics to be overlaid. For example, a 2-dimensional printed marker is placed in front of a webcam. The computer then interprets this symbol to overlay an on-screen graphic as if it were directly on top of the marker in the physical world. There have been several notable uses, most commonly in marketing   Markerless Systems: By contrast, this technological approach has given rise to \u2018mobile augmented reality\u2019, denoting use of the technology with devices such as smartphones and tablets. This method uses a combination of an electronic devices\u2019 accelerometer, compass and location data (such as the Global Positioning System \u2013 GPS) to determine the position in the physical world, which way it is pointing and on which axis the device is operating. This location data can them be compared to a database to determine what the device is looking at, and thus allows computer data/graphics to be displayed on-screen.   Applications of AR in industrial environment: 1. Data Reuse: AR applications allow more efficient reuse of existing enterprise application data. Enterprise application data can include maintenance records, service tickets, and usage logs\u2014any combination of which can be tapped into to decorate an AR experience that blends the physical and digital worlds. 2. Data Contextualization: AR applications can be used to better contextualize information across a range of industries and job functions. In manufacturing, this could mean giving linemen and plant personnel the ability to abstract backend systems information for 3D product navigation, step-by-step work instructions, and remote visual guidance. 3. Work Error Reduction: AR applications can reduce errors made by workers in various occupational tasks. Whether the error is caused by too little information, too much information, or poor quality information, AR applications place the right information and instructions at the location of the work task. 4. Workforce Multiplier: AR applications are a workforce multiplier in two ways. First, training resources are more effective and efficient. Training hours can be reduced because AR applications can include video instructions on repair/replacement tasks overlaid on the machines themselves. Second, workers can do more by themselves versus sending multiple workers or truck rolls to address the same problem or work task. 5. Safety: AR applications can also improve the safety of a work task. For instance, AR applications can include not only written instructions for removing or replacing a part on a machine, but can also include visual instructions. This assistance avoids unnecessary and possibly unsafe interaction with the machine or object.  ,  The Mobile Collaborative Robot (MCR) is developed to achieve autonomous material transportation within the Cyber Physical Factory. The MCR consists of a TM5M700 collaborative robot mounted on top of an Addverb Automated Mobile Robot (AMR). The AMR uses a LIDAR and two depth cameras to navigate using Natural Navigation. It has a payload of 150 kg. The AMR runs on Robot Operating System (ROS). The collaborative robot mounted on top of the AMR has a camera on it. The cobot can be programmed using ROS or the TMFlow software. The feed from the camera can be used for object detection. The cobot has a payload of 6 Kg. The MCR can also be controlled using a Fleet Management System. The MCR can be dispatched to various locations in the map using REST Api communication through the Fleet Management System. ,  Coming Soon.. ,  <-Robotic Welding Cell ,  Cyber Physical Factory-> ,  Short-term education and training courses will be offered along with on-line learning and assessment tools to educators and users on a chargeable basis. All the 12 technology streams will be covered. ,  The domain of Smart Manufacturing is expected to open up a new opportunity in industrial consulting, especially with regard to identification and removal of bottlenecks in current operations. The CEFC multi-disciplinary teams will undertake feasibility surveys, ROI analysis and process improvement studies for clients, prior to implementation and also post implementation. ,  Many times clients wish to test out the Proof-of-Concept on their actual production lines. The CEFC team assigned to the Client during concept formulation will undertake such site activities with a pool of industry solution providers and system integrators. ,  It is expected that Skill Councils such as Automotive Skill Council, Capital Goods Skill Council,Industrial Automation Skill Council and Tool Rooms will be creating their unique set of Role based competencies. The CEFC will offer them and their Training Partners, a gap filling option for theory and practical, and a brand-agnostic independent certification. ,  The research team will help users to bring uniqueness into their solution while providing adequate safeguards for IP protection. Users can sponsor prototyping or R&D projects as per their need. ,  IAFSM will take up research projects in consultation with industry for development of cyber physical systems for machine tools, industrial IoT, machine data cloud, machine controllers etc. to bridge the technology adoption gap in manufacturing. These research projects will be executed with clear objectives to meet focussed technology requirement for smart manufacturing and easy adoption by the industries. The developed technologies will be licensed further for commercialization to interested industries. ,  The CPS facility and the associated technologies will provide a one stop shop for users whose needs cut across multiple disciplines and who want to witness a comprehensive digital transformation before taking up for actual implementation. This will enable clients to experiment and innovate with an appropriate mix of standard and customised solutions. ,  The testing service will enable clients to test and debug problems before installing the solution at site. ,  Multi Process Robotic Cell ,  Smart Lathe Machine ,  Robotic Welding Cell ,  FSM AR Demo ,  Mechanism Kit Assembly and Disassembly ,  AR Maintenance Using HoloLens ,  Expert Maintenance System Using HoloLens ,  Cyber Physical Assembly Line ,  CP Lab Autonomy ,  Smart Mechanism Kit Realtime Values ,  PLC Kit Parts Introduction  ,  Smart Mechanism Kit Motor Replacement ,  To create user defined webpages to control and monitor different devices (Appliances,machines) present in CP lab. ,  01Remote Control and monitoring of assembly line, Compressor and MPRC ,  02Remote control and monitoring of domestic appliances (lights,fans,AC) ,  03User defined access to control the machines remotely ,  04Activity logging of users ,  05Remote Control of the training Kits to train people remotely  ,  Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available. Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology. Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are \ enabling technologies\  which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a  thing in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized.   Currently, manufacturing data are segmented, detailed and planned for a single scope, stored within the legacy systems, thus preventing the digital continuity that would let use them in the optimal way, independently from where and how they have been collected. Cyber Physical System (CPS) will instead be able to provide the needed information from the physical world while cyber-physical-collaboration environment will enable an efficient analysis, management, sharing and usage of the data and the knowledge elaborated from them and from the experience of involved people. Cyber-physical systems also represent a paradigm break from existing business and market models, as revolutionary new applications, service providers and value chains become possible.  ,  Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer\u2019s machines are up and running much faster, saving them from costly downtime.   For a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.   A secure remote connection to distributed machinery and equipment is also the basis for many new concepts and services within Industry 4.0 such as predictive maintenance, where a secure connection is established to collect data from machines, equipment or devices. The data is than analyzed and used to detect errors and possible failures at an early stage to avoid unplanned downtime. ,  Software used for AR : Here are various software we are using for the smart manufacturing purpose in AR. ,   It is an AR application designed and developed to provide a complete 3D CAD model of the FSM Smart Kits. The application uses a pre-defined Image Target and superimposes a 3D CAD of the Smart Manufacturing Training Kits. The user can visualize the smart kit without having access to the physical kit. He/She can click on the individual parts to display information about them. He/She can also get an exploded view of the CAD to understand the disassembly process. All activity of the user is also being logged on the cloud and displayed on a dashboard, this allows us to track the progress of the user. ,  It is an AR application which uses the physical FSM Smart Kit as an Image Target and adds information about their individual components. The app also allows the user to click on the individual parts and learn about them. ,  It is an AR application to help the surgeon in verifying and sensing the correctness of the bent plate. The application uses a 2D Image Target to superimpose the CAD of the bent plate on the Image Target. The surgeon will try to match the physical plate to the superimposed cad to confirm the correct bent. ,  It is an AR application to guide the surgeon for drilling correct holes in the spine to insert screws. The application uses a 3D spine as the Model Target, then superimposes the screws on the CAD. This will guide the surgeon to determine the angle and location of the screws.  ,   It is an AR application that provides a step-by-step process to assemble and disassemble the FSM Mechanism Kit. ,  It is an AR application that provides the user a 3D visualization of the Cobot joints' movements. Use sliders to move the joints and learn about the working of Cobots. ,  It is an AR application to scan the surroundings and place full-scale objects to confirm size requirements. ,  It is an AR application to get a 3D view of all the smart machines and kits at FSM. ,  It is an AR application that replicates the assembly process of the assembly line. It gives the user a complete visualization of every step of the assembly process. ,  Smart Catalogue & visualization ,  Education &  Training ,  Use as a teaching tool ,  Assess machinery before purchase ,  Maintenance Tool ,  <-Smart Manufacturing Training Kits ,  Cyber Physical Facilities-> ,  Cyber physical factory (CPF) is also known as Discrete Micro Production Facility or simply Micro Production Facility. It demonstrates the convergence of the Informational Technology (IT) and Operational Technology (OT) at micro factory level where the machines are discretely located. It has 18 varieties of machines for different manufacturing operations. The key features that this factory has: Customer Order Management Production Order Management Material Management Performance Management Customer Relationship Operations Intelligence Quality Management Error-Proofing Maintenance Management Energy Management Workforce Management Environment and Safety We have converted legacy machines into digitized smart machines like lathe machine, shearing machine. The Mobile Collaborative Robot is used for inbound logistics. The following image demonstrates the material and manufacturing process for a typical product like a hydraulic pump flange. ,  <-Mobile Collaborative Robot ,   Services-> ,  This course is designed on a Simulation platform. It\u2019s a self-paced online course where you get to learn different domains of Automation like Pneumatic, Electro-Pneumatics, PLC programming and Electrical Connections of a PLC. You will create different industrial projects and simulate them on the software during the course tenure. This course is a blend of lecture videos, Step-by-step documents, and Live sessions to give a complete idea and knowledge of Industrial Automation to you. By the end of the course, you will be ready to write a PLC program for machines with DI, DO, AI and AO. You will be able to understand and create pneumatic and electro-pneumatic circuits depending on the industrial application. ,  Pneumatic Components Introduction, Creating Pneumatic connections on software and creating an Industrial Application ,  Electro-Pneumatic Components Introduction, Creating Pneumatic and electrical connections on software and creating an Industrial Application ,  Hardware identification of PLC , PLC Programming on basic instructions, Timers, Counters, and Digital I/OsProject: Programming a PLC for an Industrial Application ,  PLC programming on compare instructions and Analog I/Os Project: Programming a PLC (with analog Inputs and Outputs) for an Industrial Application ,  Self- project where in the participants will be given an opportunity to try out the use case that they have in mind using the simulation software to simulate and test their application ,   Is this limited to any stream?  Any knowledge of PLC is required before joining the course? Is this limited to any stream?No, Anyone interested in industrial Automation can join the course.Any knowledge of PLC is required before joining the course?No. You will be starting from the scratch but having prior knowledge will help you to complete the course early. ,  No, Anyone interested in industrial Automation can join the course. ,  No. You will be starting from the scratch but having prior knowledge will help you to complete the course early. ,  Workstation 1 is one of the assembly stations. This Station stores the raw part and inspects the parts using QR code scanner and vision sensor. Valve body is transferred from one station to another in a pallet. Pallet has a RFID tag,so that next station can read the value and get the information about the body present in the pallet. PLC Used: 1769-L33-ERM,Micrologix 1400 (Rockwell) Software Used: Studio 5000,RSlogix 500 and View ME(HMI designing)  Programming: Ladder Logic Programming, Safety Sensors (IO Link master), Pneumatics and 4 Axes Motion Programming. Workstation 2 is the next assembly station. This station inserts the child part in the base part. PLC Used: BnR Software Used: Automation Studio Programming: Ladder Logic/Structured text Programming, Safety Sensors (IO Link master), Pneumatics and 3 Axes Motion Programming.  Workstation 3 is one of the assembly stations. This Station assembles all the sub parts of Cotter joint assembly depending on the type of socket body approaching the station with the help of RFID. The Gripper Picks and Places the Cotter to engage socket and spigot. PLC Used: Mitsubishi MELSEC R-Series (Rack Type) Software Used: GX Works 3  Programming: Ladder Logic Programming, Safety Sensors, Pneumatics and 7 Axes Motion Programming.   ,  AutomationAugmented RealityIIoTMachine LearningDigital Twin ,  01Demonstration & Training ,  04Flexible Assembling ,  02Customer order processing ,  03Realtime dashboard ,  An AR application to show information about FSM Smart Manufacturing Kits  An AR application that gives the user a menu with 10 options for each of the FSM Smart Manufacturing Kit. The user can visualize all 3 components, the HMI, the Electronic Kit, and the Mechanism Kit. The user can click on the different parts and get to know about the parts. The user can also initiate the CAD explode view animation.  ,  An augmented reality application to provide self-guided training on the FSM PLC Kits. The user can scan the ThingMark on the Kit to load up the experience or can FSM PLC Kit Parts Introduction 2 use model target detection by aligning the on-screen guide with the real hardware and then click on the parts to know about them ,  This course is completely designed on the Codesys platform using Soft-PLC. Throughout this self-paced online course, participants will be trained on the basics of ladder logic programming, Visualization of a Human-Machine Interface for controlling the program parameters, and Motion programming with a basic linear axis motion to rotary of single-axis motion. This course also explains the communication between the edge tier & platform tier using a data aggregator software, an interface that is created between the Soft-PLC and the data aggregator using OPC-UA Server and Client plug-in. By the end of the course, you will be ready for programming a codesys platform-based hardware with HMI features, programming motion controller with Open PLC blocks, knowledge of how OPC-UA Server-Client works and how to communicate the data of a Soft-PLC on Codesys Driver. ,  Basics of codesys software and the basic instructions used in the ladder logic programming. ,  Designing a front-end UI for the operator to operate the parameters/tags of the program in the HMI. ,  Overview of function block programming, explaining how the single-axis linear system and the single-axis rotary index system with multiple stations. ,  Extracting the data out of the PLC to control and monitor the application remotely within the network. ,  Self- project where in the participants will be given an opportunity to try out the use case that they have in mind using the Soft-PLC along with HMI screens and remote monitoring. ,  Minimum hardware requirement1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn\u2019t run on macos or linux ,  1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn\u2019t run on macos or linux ,  PrerequisitesNo prerequisites are required ,  No prerequisites are required ,   Laptop/ PC minimum requirements  I don't know what PLC is should I join this course?  Is this limited to any Stream? Laptop/ PC minimum requirements?4GB Ram, i3 processor with windows 8 or above version.I don't know what PLC is should I join this course?Yes, you will get a chance to learn more about it.Where can I find The Economist?Its not limited, anyone with Automation interest can join this course. ,  4GB Ram, i3 processor with windows 8 or above version. ,  Yes, you will get a chance to learn more about it. ,  Its not limited, anyone with Automation interest can join this course. ,  Smart trainer kits are used to train industry and academic people and make them aware about Industry 4.0 technologies. It is a combination of three independent hardware accessories- PLC Trainer Kit, HMI Kit and Mechanism Kit. All these kits demonstrate the industry 4.0 technologies in the best possible way. These were designed, manufactured and integrated in FSM. Modules/Trainings covered in these kits: PLC Programming, Network Protocols HMI Designing Data Logging  Dashboard Designing, NodeRed IIOT gateways and platforms Augmented Reality Machine Learning Smart Sensing Motion configuration and programming Vision System   ,  This is one of the kits of the smart manufacturing trainer module. This kit has the brain of  the other two kits (HMI Kit and Mechanism Kit) as programming is done in this kit. It has a PLC for programming and digital inputs/outputs, analog inputs/outputs to connect the required sensors with the PLC. It has an IO link module to connect the smart sensors, so alongwith the 24VDC or analog input this module communicates the sensor status with the PLC. Data logging and data collection is one of the critical steps for IIOT  and for that we have integrated IOT gateway and other software platforms in this kit. To connect sensors with the PLC, banana connectors are used. It can communicate on a number of network protocols and it has the feature to communicate with multiple HMIs and Servo drives. In programming kit we have PLC hardwares from Rockwell, Siemens, Schneider, Omron, Delta, Turck and Mitsubishi for programming and various softwares. NUC is also integrated in the kit so that it can be used as CPU for softwares and PLC programming. So, this kit is more than sufficient to demonstrate the features of the PLC, gateways and software platforms (Data Logging, Augmented Reality ,IIOT gateways and platforms), Smart Sensing . ,  Human Machine Interface is used to monitor and access the PLC data. Screens can be created depending on the application.This is the second kit of the smart manufacturing trainer module. It has HMI to display the screens and monitor the PLC tags. The hardware buttons and LEDs as well which are connected with the PLC trainer kit using banana connectors, mostly like plug and play. These are used to test the PLC program and as input and output in the PLC program. HMIs used in these kits are smart HMIs which have smart features like OPC UA server/client, MQTT compatibility, user authentication on the screens, act as an edge device etc. So, kit is used to visualise the PLC logic. Additionally it demonstrates the HMI Designing and Dashboard designing feature. ,  This is the third kit of the smart trainer modules used to demonstrate a working manufacturing system. It has a servo motor and drive to rotate an indexing table. Along with the motion feature, it has four stations which can be used to demonstrate multiple operations. Each station has smart inductive sensors to sense the part's presence which consists of the distance and proximity sensors, servo drive, test product, RFID Reader, Solenoid, USB Camera mounted on them.Station 1: It has a RFID controller used to read and write the tag values of a product. It communicates with IO Link master (present in PLC trainer kit) and IO link master communicates the data with the PLC. So, RFID operation and use case is demonstrated. Station 2: This station consists of a distance sensor used to calculate the height or distance of the product. It is a smart sensor and communicates with IO Link master. So, along with the analog value it communicates it\u2019s status to the PLC as well. Verification of product dimension is also demonstrated. Station 3: This station has a vision camera installed on it. This camera is used for image processing and product testing. Sorting operation is performed using the image processing feature. Station 4: This station has a solenoid which is directly connected with the digital output of the PLC. Extension and retraction of the solenoid is programmed in the PLC for this operation.This mechanism kit is used to demonstrate and train motion configuration and programming, Vision System, Smart Sensing, Machine Learning, Augmented Reality.  ,  Programming Kit ,  HMI Kit ,  Mechanism Kit ,  It is showing the working of Smart Manufacturing Kit ,  Education & Training ,  Device Services ,  Research & Skill Certification  ,  Software-> ,  Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available. ,  The digital factory concept offers an integrated approach to enhance the product and production engineering processes and simulation is a key technology within this concept. ,  Until now, industrial robots always worked separately from humans in specially safeguarded protected spaces. Smart manufacturing has broken down this barrier with a new generation of collaborative industrial robots. ,  A key component in making Industry 4.0 a reality are machines that can produce the desired components faster, more flexibly and more precisely than ever before. Also Consumers want products that reflect their individuality. ,  Augmented reality is the integration of digital information with the user's environment in real time. Unlike virtual reality, which creates a totally artificial environment, augmented reality uses the existing environment and overlays new information on top of it. ,  Networked production and control processes in complex machine environments determine the industrial future and make Industry 4.0 possible in the first place. Smart Sensors already today support dynamic, real-time-optimized, and self-organized industry processes. ,  Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. ,  The world of automation is merging with the IT world. Safety & security is an important prerequisite for the function of Industry 4.0 systems, which in contrast to traditional production plants have interfaces to their environment. ,  An Augmented reality application that allows the user to understand about the mechanism kit and scan the image target and place a CAD model of the Mechanism Kit to understand it clearly. The user can create a new account and can access the application using the registered email id and password. If the user forgets the password associated with the registered email id, they can use the forgot password option and can create a new password for the registered email id. The user can the access the features of the application by using the main menu FSM Mechanism Kit Assembly and Disassembly 2 where different buttons are linked with the assembling and disassembling. The user can then play the animations of the screws, Allen Keys, screwdrivers, and the parts of mechanism kit. The steps can be played multiple times as per the requirements. The user can switch between the steps using the next and previous buttons. ,  Cloud DatabaseAugmented Reality ,  03Remote Maintenance ,  Mandated Guidelines  Awareness campaigns on Industry 4.0 Training for master trainers Active participation provisions for start-up / incubators Hand-holding of SMEs to plan and implement relevant Industry 4.0 projects to be done through consultancy services on a chargeable basis Collaborating with neighborhood universities for student training/internship programs Involving industry in SPV members model for sustainability Participating in a Government-formed platform for Industry 4.0 To make adequate provisions for e-waste management Involving as many clusters of 'capital goods' as possible  ,  At FSM we are committed to demystify the smart manufacturing technologies by learning ourselves and then transfer the learning through various training programs to industry and academicians. We are engaging researchers to indigenously develop technologies to make it affordable by Indian MSME and facilitate adoption of Industry 4.0. Our state of art cyber physical lab showcase technologies available across globe through our strong partnership with the automation industries. ,  With a far-sighted approach, the Government of India has supported Automation Industry and IIT Delhi in creation of advanced engineering and software facilities for smart manufacturing.  Prospective integrators, consultants and end users can use our training and tryout services to validate their proof-of-concept before scaling up. We help demystify and de-risk your investments in Industry 4.0 ,  Co-Creator ,  Dr. Sunil JhaDirector ,  Mr. Dilip SawhneyDirector ,  Dr. Anil WaliDirector ,  Sunil MehtaDirector ,  Mr. Anup WadhwaDirector Automation Industry Association ,  Mr. Ravi AgarwalDirector ,  The digital factory concept offers an integrated approach to enhance the product and production engineering processes and simulation is a key technology within this concept.  Simulation helps to create digital models of products, production systems, logistic systems so that one can explore a system\u2019s characteristics and optimize its performance. These digital models have the ability to run experiments and what-if scenarios without disturbing existing production systems or \u2013 when used in the planning process \u2013 long before the real production systems are installed. Extensive analysis tools, such as bottleneck analysis, statistics and charts give us an opportunity to evaluate different manufacturing scenarios. The results provide us with the information needed to make fast, reliable, smarter decisions in the early stages of production planning. In addition, we can also optimize material flow, resource utilization and logistics for all levels of plant planning from global production facilities, through local plants, to specific lines. This helps to fulfil the user\u2019s need to deliver just-in-time (JIT) or just-in-sequence (JIS).  Benefits  Increase existing system productivity  Optimize resource consumption and re-use  Optimize systems for reduced energy consumption  Shorten new product introduction, time-to market, and time-to-volume  Improve production layout and minimize investments  Ensure that machines and equipment are in the right place  Ensure that sufficient material handling equipment is available  Optimize buffer sizes  Ensure that product handling is kept to a minimumApart from advance simulation another disruptive innovation is the concept of digital twin. With the growing deployments of Internet of Things (IoT) systems, the importance of the concept of a digital avatar of a physical thing has gathered significant interest in the recent years. A digital twin can be defined, fundamentally, as an evolving digital profile of the historical and current behavior of a physical object or process that helps optimize business performance.A digital twin enables flexibility in manufacturing to reduce time needed for product design, manufacturing process and system planning and production facility design. A digital twin improves quality and even supports new business models that offer opportunities for small- to mid-size companies to expand and bring more high-tech capability into their shops. Digital twins will help all companies become more flexible, reduce time to market, reduce cost, improve quality and increase productivity at all levels of the organization. Digital twin business values: 1. Quality  Improve overall quality  Predict and detect quality trend defects sooner  Control quality escapes and be able to determine when quality issue started 2. Warranty cost and services  Understand current configuration of equipment in the field to be able to service more efficiently  Proactively and more accurately determine warranty and claims issues to reduce overall warranty cost and improve customer experiences 3. Operations cost  Improve product design and engineering change execution  Improve performance of manufacturing equipment  Reduce operations and process variability 4. Record retention and serialization  Create a digital record of serialized parts and raw materials to better manage recalls and warranty claims and meet mandated tracking requirements 5. New product introduction cost and lead time  Reduce the time to market for a new product  Reduce overall cost to produce new product 6. Revenue growth opportunities  Identify products in the field that are ready for upgrade   Improve efficiency and cost to service productThese digital twins use data from sensors that are installed on physical objects to represent their near real time status, working condition or position. Nonetheless, advances in computer science have made it possible to broaden the scope of the digital twin to include many more capabilities, information, inputs and outputs. It also helps you develop and introduce new products to the market much faster than ever.    ,  Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment.As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory\u201d. ,  Registration Open ,  Last day to register ,  Short Listing ,  Orientation ,  Level 1 Learning Ends ,  Level 2 Technology Stream Learning Ends ,  Final Selection and Project Allocation ,  Orientation 2nd Programme & Internship Starts ,  Internship Ends ,  Registration  Short Listing Level 1 Learning Level 2 Technology Stream Learning Final Selection Project Allocation Internship Commencement  ,    Fill out the online registration form Shortlisting based on the eligibility criteria Level 1: Learning modules (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023. Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship.   Level 2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023. Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors.  Final Selection and Project Allocation based on the Level 2 performance and preferences These are full-time internships and require 48 hours per week  ,   AR-based fault diagnosis of RWC Implementing AR for complete Demonstration of Robotic Welding Cell Unity ROS Cobot FSM AMR AR FSM Interactive lab introduction  ,  02Digital Twin ,   Digital Twin of Robotic Welding Cell for process optimization and Predictive Maintenance. Digital twin for Advanced Milling Machine Centre  ,  03Industrial Automation ,   CP Lab Autonomy Phase - 2 Multi Process Robotic Cell Simulation on a simulation platform Robotic Welding Cell Simulation on a simulation platform Training Course on Rockwell Kit Training Course on Siemens Kit  ,  04Industrial IoT ,   Implementing IIoT on CPL Autonomy Implementing IIoT for Plant Owner on Robotic Welding Cell  ,  05Machine Learning ,   Remaining Usable Life Estimation (NASA Turbine Dataset) Power Line Fault Detection Computer Vision to detect defects in PCB Power Line Fault Detection Steel Defect Detection Using Computer Vision  ,  06Manufacturing Execution System ,   Digital quality report generation for 3d prints. Admin & customer pages for Order Management Portal (RWC)  ,  07Real Time Dashboard ,   Visualization of Multi-Process Robotic Cell Interactive Portal for RWC: Supervisor  ,  08Robotics ,   MPRC OLP Generation using Sim4.1 for a product Use of AMR-Cobot to place pallet ROS Integration of AMR Integration of computer vision with Cobot Computer Vision Integration with Cobot  ,  0Project Domains ,  0No. of projects done ,  0Candidates completed the Internship ,  0Candidates applied ,  0Candidates achieved Level 1 proficiency ,  0Candidates were awarded with prize of INR 5000/- for Best Project Award ,  0Candidates were shortlisted ,  0Candidates were selected for presentation before Jury Members ,  0Research papers given merit certificate  ,  01Augmented Reality ,  02Industrial Automation ,  03Industrial IoT ,  04Machine Learning ,  05Real Time Dashboard ,  06Robotics ,  Here are some videos of our interns presenting their final projects in front of the jury members and their mentors. Jury members were invited from different industries with their expertise in respective domains. In the presentation, our interns explained their whole journey and gave the detailing of the projects. After reviewing all these projects, the top 10 interns got awarded the best project award in their domain and a cash prize of  5000. ,  The term safety denotes the functional safety of machinery or, put another way, the protection of people and the environment against threats that can proceed from machinery. Safety demands that residual risks arising from a plant or machine do not exceed acceptable values. This includes hazards to the plant environment (e.g. environmental damage) as well as hazards within the plant or machine (e.g. people inside the plant). One option for the worst case is simply to interrupt the energy supply straight away and bring the machine to a hard stop. The traditional way of providing scope for this is by means of special safety wiring and components such as safety relays. Because this approach is very much hardware-based and therefore static, it is not particularly suitable for intelligent manufacturing processes where plant layouts continually need to be changed. A hard shutdown is generally associated with further disadvantages, whether these involve loss of productivity, extended downtimes due to more complex recommissioning procedures or a restriction in the machine\u2019s operating and maintenance concept.   An alternative is offered by dynamic safety concepts based on an integrated view of changing automation processes and functional safety requirements. This changes the view of safety itself; it is regarded less as a hardware characteristic and more as a cross-device function. This approach allows processes to be operated in a safely controlled manner without any need to interrupt them immediately every time a fault occurs. But the dynamic approach can only be implemented efficiently if functional safety is built into automation projects from the moment they are planned. ,  An Augmented Reality application to show the Real-time Values of the 4 stations of the FSM Smart Mechanism Kit by connecting with KEPServer and getting tag values ,  Augmented RealityIIoT ,  02Realtime Monitoring ,  Robotic Welding Cell demonstrates a flexible manufacturing cell meaning it has an inventory of its own, material handling system, variety of parts with dynamic-order.This welding cell can be a part of production line and act as standalone system.Use of single or multiple robots may involve use of a PLC also.This cell uses a saftey PLC.This cell has all these features and the basis of selection can be done in a way that the cell will be programmed, utilization of peripheral devices (proximity sensors, pneumatic cylinders), safety devices and Automatic Identification & Data Capturing (AIDC) devices like RFID, vision camera etc. The cell has following level of flexibility but the decision can be made on the basis of the flexibility required as per the following parameters:1. Variety of parts for simultaneous production 2. Change of schedule3. Ready for new partWorkstations and layout design :Workstations in the cell are as per the following given applications and subject to following parameters : 1. Load/Unload stations for the cell: It is the area where the raw materials are incepted in/out of the cell.2. Inventory storage area: An inventory storage where the storage of the raw material is kept.3. Inspection area: An area where the material post operations are inspected for quality. This is kept as a separate unit away from the cell but it can be an integral part of the cell itself.4. Operations area: An area where the operations like welding is performed. In the cell , this area is more of a workspace since two robots are responsible for performing welding on the material. And it can be anywhere in the robot's workspace. 5. Layout design: It is an integral part before identifying the robots and environment preparation and also plays a key role in safety of the cell.CAD Design & Simulation: The 3D visualization and detailing of the cell is done using a designing software. The robotic path simulation software asists in defining the best possible, collision free environment that is going to be the basis of the layout design and all other hardware that are going to be installed in the cell. Apart from designing & fabrication of the cell it helps in performing the robot simulations to generate offline program, collision free environment (this is how the level of flexibility achieved) for robots at operating stage and generating Augmented Reality applications. ,  The cell programming can be done in atleast two ways. Online programming where  teach pendant is used and other is offline programming where the simulation is done in a separate software like RoboDK and KUKA Sim4. This helps in visualising and performing collision free path generation for both the robots. The video below demonstrates the offline programming method where we can see two robots performing indexed type motion. The offline programming helps in virtual commissioning and without impacting the production.  ,  Simulation & Testing ,  Job work & research ,  Prototyping ,  Customer Order Proessing ,  Site Integration ,  Machine-as-a-service ,  <-Multi Process Robotic Cell ,  Mobile Collaborative Robot-> ,  Smart sensors are a prerequisite for creating the best possible basis for a future-oriented automation system. This is because the smart factory needs data that can principally only be provided by smart, intelligent and communication-enabled sensors. Communication-enabled means being able to exchange sensor data with a machine controller or a cloud-based application. Thus, for example, sensor parameters are automatically adapted to new production orders within seconds. Or a light barrier detects contamination of its optics and reports this directly to the control center.   Smart Sensors generate and receive data and information which goes beyond traditional switching signals or measured process parameters. They therefore enable substantial increases in efficiency, more flexibility, and better planning security for predictive plant maintenance. Depending on the requirement, Smart Sensors cover up to four dimensions of Smart Sensor technology.   1. Enhanced sensing 2. Efficient communication 3. Diagnostics 4. Smart tasks Just like smart sensors, smart actuators also have intelligence built into them. Some of the smart features possessed by different types of actuators are:   Open standards: Multi-protocol communication interface allow universal operation with diverse, ethernet-based communication protocols Web-based commissioning and diagnostics via integrated web server Comprehensive access to actuator data (e.g. torque or power), via web connector for preventive maintenance or database interfacing and data analysis Inbuilt smart sensors for capturing data Flexible configuration according to requirement Comprehensive condition monitoring for fast identification of critical conditions and comfortable analysis Simple connection to higher-level data systems without additional hardware Workpiece-dependent parameters setting guaranteeing quality results Integrated control system with browser-based operating system allows for information sharing across multi vendor data systems  ,  Robotic Welding Cell demonstrates a flexible manufacturing cell meaning it has an inventory of its own, material handling system, variety of parts with dynamic-order.This welding cell can be a part of production line and act as standalone system.Use of single or multiple robots may involve use of a PLC also.This cell uses a saftey PLC.This cell has all these features and the basis of selection can be done in a way that the cell will be programmed, utilization of peripheral devices (proximity sensors, pneumatic cylinders), safety devices and Automatic Identification & Data Capturing (AIDC) devices like RFID, vision camera etc. The cell has following level of flexibility but the decision can be made on the basis of the flexibility required as per the following parameters:1. Variety of parts for simultaneous production2. Change of schedule3. Ready for new partWorkstations and layout design :Workstations in the cell are as per the following given applications and subject to following parameters : 1. Load/Unload stations for the cell: It is the area where the raw materials are incepted in/out of the cell.2. Inventory storage area: An inventory storage where the storage of the raw material is kept.3. Inspection area: An area where the material post operations are inspected for quality. This is kept as a separate unit away from the cell but it can be an integral part of the cell itself.4. Operations area: An area where the operations like welding is performed. In the cell , this area is more of a workspace since two robots are responsible for performing welding on the material. And it can be anywhere in the robot's workspace. 5. Layout design: It is an integral part before identifying the robots and environment preparation and also plays a key role in safety of the cell.CAD Design & Simulation:The 3D visualization and detailing of the cell is done using a designing software. The robotic path simulation software asists in defining the best possible, collision free environment that is going to be the basis of the layout design and all other hardware that are going to be installed in the cell. Apart from designing & fabrication of the cell it helps in performing the robot simulations to generate offline program, collision free environment (this is how the level of flexibility achieved) for robots at operating stage and generating Augmented Reality applications. ,  AutomationIIoTRoboticsAugmented Reality ,  01Machine as a service ,  02Customer Order Proessing ,  03Job work and research ,  3D printing has the capability to be used to respond too many of the world\u2019s changing megatrends. The physical object is made from a three-dimensional digital model, typically by laying down many thin layers of a material in succession. With 3D printing it is possible to go directly from digital design data to a final part with no intermediate production steps. 3D printing technologies therefore eliminate the need for tooling and the associated capital investment. The result is that companies that adopt 3D printing can disrupt the traditional economies of scale, by allowing cost-effective production of single-unit or low-volume batches with low-volume part production, products can be customized to local markets, or even to individual customer tastes, driving adoption within industries as diverse as fashion, health care and automotive. Moreover, with the ability to print on demand, businesses also have the opportunity to eliminate inventory and cut aftermarket lead-times by providing digital spare-parts catalogues that can be printed when needed. The smart companies of the future will be those that have a clear strategy for how and where 3D printing fits within their supply chain and their value chain. They will also understand the business drivers to adoption.   From the student\u2019s study to the professional designer\u2019s office, from the dental laboratory to the jewellery retailer, from the aerospace factory to the hospital basement, 3D printers have become invaluable business tools. Applications and reasons are as diverse as the users. What links all these applications and users is one underlying ability: to transition 3D information digitally and seamlessly from the virtual world to the real world with nothing but a computer and a3D printer \u2013 from bytes to bits.   The benefits of 3D printing today go way beyond just making models and prototypes. 3D printing is becoming a way of making components, systems and products sold across the supply chain, from the bracket in the aircraft door to the dental aligner in the teenager\u2019s mouth. It can also be used for the manufacturing of tools, jigs and fixtures in addition to rapid prototyping. ,  An AR application designed to run on HoloLens 2 to demonstrate the entire sequence for Motor Replacement in the FSM Smart Mechanism Kit.The application uses a model target to detect the real mechanism kit and then superimpose the AR kit.The application will guide in tool selection and then show the steps to replace the motor.Voice updates are given to the user with each step.The user can also control the application sequence using voice commands. ,  Augmented RealityMicrosoft HoloLens 2 ,  01Self-Guided Training ,  04Self-Guided Maintenance ,  02Action Latency Reduction ,  03Insight Latency Reduction ,  IITD-AIA Foundation For Smart Manufacturing presents a comprehensive Learning Module on  Industrial Automation and IoT using soft-PLC Program, visualize, execute remote monitoring, motion programming and more using soft-PLC on the upcoming Learning Module. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha ,  IITD-AIA Foundation For Smart Manufacturing presents a 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha ,   ,  IITD-AIA Foundation For Smart Manufacturing presents 4 hours free awareness course on Industrial Automation Learning. This is a supplement course helpful for the upcoming 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing ,  or email us oninfo@iafsm.in ,  Smart Lathe Machine project is about retrofitting of Sensors and PLC for Digitizing the Cutting Parameters. Additionally for the monitoring of Power Consumption, the energy meter is in place to find out the power related parameters. The idea is to digitize the data and add operator and machine safety interlocks to avoid accidents in the shop floor. Also, the digitized data will be used in implementing Industrial IoT dashboards for monitoring purposes. The cutting parameters along with energy data is then used for training the machine model to predict the roughness or the remaining useful life of the tool depending on the vibration signals acquired through the vibration sensor. The Augmented Reality experience is created around the machine for giving basic nomenclature of Lathe machine. The animation of each process is also part of the experience. ,  AutomationIIoTMachine LearningAugmented Reality ,  01Augmented Reality for guiding the students learn the different components of the machine ,  04Real-Time Health Monitoring of Assets in Factory ,  02Augmented Reality with guided  exercises for maintenance. ,  03Machine learning for predicting the tool wear. ,  Contact us today for implementing ,  Call us for any query011-26582053, 8076197190 ,  or email us oninfo@iafsm.in   We\u2019ve detected that JavaScript is disabled in this browser. Please enable JavaScript or switch to a supported browser to continue using twitter.com. You can see a list of supported browsers in our Help Center. ,  Help Center ,   Terms of Service Privacy Policy Cookie Policy Imprint Ads info       \u00a9 2023 X Corp.      ,   Level 2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023. ,  Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors. ,  Short term education training offered along with online learning and assessment tool. Participants are engaged for approximately 100 hours in different technologies that include Automation, Industrial IoT, Augmented Reality, Robotics, Digital Twin and etc. The learnings are divided into Awareness Level, Learning Level and Expert Level. ,  CPS facilities and the associated technologies will provide for one-stop shop for users whose needs are cut across multiple disciplines and who wants to witness comprehensive digital transformation before taking up the actual implementation. ,  We are providing following services: ,  Indian Institute of Technology Delhi (IITD) and Automation Industry Association (AIA) joined hands in May, 2017 to set up a center under auspices of Samarth Udyog - a project of the Ministry of Heavy Industries (MHI). The center is called Foundation for Smart Manufacturing (FSM), and it helps supports and develops technologies for right understanding and implementation of concepts of smart manufacturing. FSM is a demo-cum-experience facility in North India, vests in extensive skill building, MSME consultancy, multi-academia partnerships and research which will give a huge fillip to technology aided competitiveness of Indian manufacturing. The project will imbibe technologies from Europe, Japan, USA and India. 'Producing Smarter' with Technologies. FSM has been launched specifically for implementation of smart manufacturing in India which it undertakes to do effectively by: Awareness Building, Prototyping, Simulation and testing Services, Consulting Services, Site Integration Services, Education and Training, Skills Certification, Job work and Research. ,  Director ,  info@iafsm.in ,  FSM is providing an exciting opportunity for 100 individuals to participate in a summer internship program. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of \ Smart Factory,\  which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects. ,  FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0 ,     "

In [56]:
# def divide_into_chunks(context, chunk_size=512, stride=50):
#     chunks = []
#     total_words = len(context.split())
#     start = 0

#     while start < total_words:
#         end = min(start + chunk_size, total_words)
#         chunks.append(context.split()[start:end])
#         start += stride

#     return chunks

In [3]:
import torch
#from transformers import BertTokenizer, BertForQuestionAnswering

def divide_into_chunks(context, question, chunk_size=512):
    question_tokens = tokenizer(question, return_tensors='pt')
    question_len = len(question_tokens.input_ids[0])

    chunks = []
    total_words = len(context.split())
    start = 0

    while start < total_words:
        # Calculate the remaining space in the chunk for context.
        remaining_space = chunk_size - question_len

        # Ensure the remaining space is positive.
        if remaining_space <= 0:
            break

        end = min(start + remaining_space, total_words)
        chunks.append(context.split()[start:end])
        start += remaining_space

    return chunks

# Example large context and question (Replace these with your actual context and question).
#large_context = "Your large context goes here..."
question = "What is FSM?"

# Load the tokenizer and model (we'll use BERT for question answering).


# Divide the large context into chunks.
chunks = divide_into_chunks(context, question, chunk_size=512)

answers = []

# Iterate through each chunk and get the answer using the transformer model.
for chunk in chunks:
    # Combine the chunk with the question.
    input_text = question + " " + " ".join(chunk)

    # Tokenize the combined text.
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)

    # Get the start and end positions of the answer using the model.
    inputs = tokenizer.encode_plus(input_text, return_tensors='pt',truncation=True, max_length=512)
    input_ids = inputs['input_ids']
    tokenizer.decode(inputs.input_ids[0])
#print(len(input_ids))
    attention_mask = inputs['attention_mask']
    # Get the start and end positions of the answer using the model.
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)

    # Find the answer span within the chunk.
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits) + 1  
    print(start_idx,end_idx)
    # Convert token indices back to answer text.
    answer = tokenizer.decode(inputs.input_ids[0][start_idx:end_idx])
    print(answer)
    answers.append(answer)
    all_answers_str = " ".join(answers)





NameError: name 'tokenizer' is not defined

In [63]:
import torch
#from transformers import BertTokenizer, BertForQuestionAnswering

def divide_into_chunks(context, question, chunk_size=510):
    question_tokens = tokenizer(question, return_tensors='pt',padding=True)
    input_ids = question_tokens['input_ids']
    tensor1=input_ids[0]
    tensor2=torch.Tensor([102])
    
    question_tokens['input_ids'] = torch.cat((tensor1, tensor2), dim=0)

    
    
    #print(tokenizer.decode(input_ids[0]))
    attention_mask = question_tokens['attention_mask']
    tensor_mask1=attention_mask[0]
    tensor_mask2=torch.Tensor([1])
   
    question_tokens['attention_mask']= torch.cat((tensor_mask1, tensor_mask2), dim=0)
    question_len = len(question_tokens.input_ids)
    question_len=question_len+1

    chunks = []
    total_words = len(context.split())
    start = 0

    while start < total_words:
        # Calculate the remaining space in the chunk for context.
        remaining_space = chunk_size - question_len

        # Ensure the remaining space is positive.
        if remaining_space <= 0:
            break

        end = min(start + remaining_space, total_words)
        chunks.append(context.split()[start:end])
        start += remaining_space

    return chunks

# Example large context and question (Replace these with your actual context and question).
#large_context = "Your large context goes here..."
question = "What is Cpec?"

# Load the tokenizer and model (we'll use BERT for question answering).


# Divide the large context into chunks.
chunks = divide_into_chunks(context, question, chunk_size=510)

answers = []

# Iterate through each chunk and get the answer using the transformer model.
for chunk in chunks:
    # Combine the chunk with the question.
    input_text = question + " " + " ".join(chunk)
     
    # Tokenize the combined text.
    #inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)
    
    # Get the start and end positions of the answer using the model.
    inputs = tokenizer.encode_plus(input_text, return_tensors='pt',truncation=True, max_length=510)
    
    # Get the start and end positions of the answer using the model.
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)

    # Find the answer span within the chunk.
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits) + 1  
    print(start_idx,end_idx)
    # Convert token indices back to answer text.
    answer = tokenizer.decode(inputs.input_ids[0][start_idx:end_idx])
    print(answer)
    answers.append(answer)
    all_answers_str = " ".join(answers)

tensor(6) tensor(65)
at fsm we provide various client services and training under the domain of smart manufacturing. the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions., consulting,
tensor(6) tensor(65)
internship projects : after successful completion of level - 2, interns will be allocated projects based on domain preference, level - 2 performance and interaction with the mentors. final selection and project allocation based on the level 2 performance and preferences these are full - time internships and require 48 hours per week
tensor(6) tensor(65)
will be rejected. 4. hex - nut assembly station : - hex nut will be assembled on the body in this station. 5. packaging station : assembled valve bodies will be placed in a box and packaging is done. a printing head will print the required details ( type of bodies,
tensor(6) tensor(

In [60]:
# question="What is FSM?"
# input_text = question  + " ".join(all_answers_str)

# # Tokenize the combined text.
# #inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)

# # Get the start and end positions of the answer using the model.
# inputs = tokenizer.encode_plus(input_text, return_tensors='pt',truncation=True, max_length=512)
# input_ids = inputs['input_ids']
# #print(len(input_ids))
# attention_mask = inputs['attention_mask']
# # Get the start and end positions of the answer using the model.
# with torch.no_grad():
#     output = model(input_ids, attention_mask=attention_mask)

# # Find the answer span within the chunk.
# start_idx = torch.argmax(output.start_logits)
# end_idx = torch.argmax(output.end_logits) + 1  
# print(start_idx,end_idx)
# # Convert token indices back to answer text.
# answer = tokenizer.decode(inputs.input_ids[0][start_idx:end_idx])
# print(answer)


tensor(6) tensor(334)
a n d a d m i n i s t r a t i v e e n v i r o n m e n t. f s m s k i l l s i s a h o l i s t i c p l a t f o r m f o r p r o v i d i n g i m m e r s i v e t r a i n i n g t h i s v i a t h e r o b o t c o n t r o l l e r. c o l l a b o r a t i v e r o b o t s w o r k s h a n d i n h a n d w i t h t h e o p e r a t o r, t h e r e b y e n a b l i n g w i t h 3 / 2, 5 / 2, 5 / 2 d o u b l e s o l e n o i d, 5 p r o v i d i n g f o l l o w i n g s e r v i c e s : 3 d p r i n t i n g s m a r t m a n u f a c t u r i n g t r a i n i n g k i t s f o r i n d u s t r i a l a u t o m a t i o n l e a r n i n g m i g w e l d i n g ( i n c l u d i n g ).


In [61]:
# for i in answers :   
#     input_text = question  + " ".join(i)

#     # Tokenize the combined text.
#     #inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)

#     # Get the start and end positions of the answer using the model.
#     inputs = tokenizer.encode_plus(input_text, return_tensors='pt',truncation=True, max_length=512)
#     input_ids = inputs['input_ids']
#     #print(len(input_ids))
#     attention_mask = inputs['attention_mask']
#     # Get the start and end positions of the answer using the model.
#     with torch.no_grad():
#         output = model(input_ids, attention_mask=attention_mask)

#     # Find the answer span within the chunk.
#     start_idx = torch.argmax(output.start_logits)
#     end_idx = torch.argmax(output.end_logits) + 1  
#     print(start_idx,end_idx)
#     # Convert token indices back to answer text.
#     answer = tokenizer.decode(inputs.input_ids[0][start_idx:end_idx])
#     print(answer)


tensor(35) tensor(92)
f s m s k i l l s i s a h o l i s t i c p l a t f o r m f o r p r o v i d i n g i m m e r s i v e t r a i n i n g
tensor(6) tensor(97)
t h i s v i a t h e r o b o t c o n t r o l l e r. c o l l a b o r a t i v e r o b o t s w o r k s h a n d i n h a n d w i t h t h e o p e r a t o r, t h e r e b y e n a b l i n g
tensor(6) tensor(37)
w i t h 3 / 2, 5 / 2, 5 / 2 d o u b l e s o l e n o i d, 5
tensor(6) tensor(124)
p r o v i d i n g f o l l o w i n g s e r v i c e s : 3 d p r i n t i n g s m a r t m a n u f a c t u r i n g t r a i n i n g k i t s f o r i n d u s t r i a l a u t o m a t i o n l e a r n i n g m i g w e l d i n g ( i n c l u d i n g
tensor(8) tensor(76)
c o m b i n i n g a r a n d t h e i o t a l l o w s e m p l o y e e s, s u c h a s f i e l d m a i n t e n a n c e w o r k e r s, t o
tensor(6) tensor(95)
f u n c t i o n s. i n m a n u f a c t u r i n g, t h i s c o u l d m e a n g i v i n g l i n e m e n a n d p l a n t p e r s o n n e l t h e a b i l

In [ ]:
# import torch
def context_conversion(input):
  
    tokens=tokenizer.encode_plus(input,add_special_tokens=False,return_tensors="pt")
   

  
    input_ids_chunks=tokens['input_ids'][0].split(509)
    mask_chunks=tokens['attention_mask'][0].split(509)
    
    input_ids_chunks= list(input_ids_chunks)
    mask_chunks= list(mask_chunks)
    
    tensor1=input_ids_chunks[0]
    tensor2=torch.Tensor([102])
    given_word=torch.Tensor([1029])
    index = (tensor1 == given_word).nonzero()
    index = index[0]
    tensor1_part1 = tensor1[:index+1]
    tensor1_part2 = tensor1[index+1:]
    input_ids_chunks[0] = torch.cat((tensor1_part1, tensor2, tensor1_part2), dim=0)

    
    tensor_mask1=mask_chunks[0]
    tensor_mask2=torch.Tensor([1])
    index = index[0]
    tensor1_mask_part1 = tensor_mask1[:index+1]
    tensor1_mask_part2 = tensor_mask1[index+1:]
    mask_chunks[0] = torch.cat((tensor1_mask_part1, tensor_mask2, tensor1_mask_part2), dim=0)

   

    chunksize=512
    # input_ids_chunks= list(input_ids_chunks)
    # mask_chunks= list(mask_chunks)
    
    
    for i in range(len(input_ids_chunks)):
        input_ids_chunks[i]=torch.cat([torch.Tensor([101]),input_ids_chunks[i],torch.Tensor([102])])
        mask_chunks[i]=torch.cat([torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])])
        
        pad_len=chunksize-input_ids_chunks[i].shape[0]
        if pad_len>0:
            input_ids_chunks[i]=torch.cat([input_ids_chunks[i],torch.Tensor([0]* pad_len)])
            mask_chunks[i]=torch.cat([mask_chunks[i],torch.Tensor([0]* pad_len)])
    

    input_ids=torch.stack(input_ids_chunks)
    attention_mask=torch.stack(mask_chunks)
    
    input_dict={
        "input_ids":input_ids.long(),
        "attention_mask":attention_mask.int()
    }
    return input_dict 

In [ ]:
question="what is Hex-nut Assembly Station?"

inputs = context_conversion(question, context)
print(inputs)
input_ids = inputs['input_ids']
print(len(input_ids))
attention_mask = inputs['attention_mask']
tokenizer.decode(input_ids[0])


with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)
print("hey")
answer_start = torch.argmax(output.start_logits)
print(answer_start)
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)
answer_end = torch.argmax(output.end_logits)
answer_end_j=int(answer_end%512)
print(answer_end)


# # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))   
# # print(answer) 


TypeError: context_conversion() takes 1 positional argument but 2 were given

In [ ]:
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)

answer_end_j=int(answer_end%512)
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))   
print(answer) 

In [ ]:
question="what is CEFC?"

inputs = context_conversion(question, context)
print(inputs)
input_ids = inputs['input_ids']
print(len(input_ids))
attention_mask = inputs['attention_mask']
tokenizer.decode(input_ids[0])


with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)
print("hey")
answer_start = torch.argmax(output.start_logits)
print(answer_start)
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)
answer_end = torch.argmax(output.end_logits)
answer_end_j=int(answer_end%512)
print(answer_end)

In [ ]:
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)

answer_end_j=int(answer_end%512)
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))   
print(answer) 

In [ ]:
# def get_answers(question,context):
#     inputs = context_conversion(question, context)
#     input_ids = inputs['input_ids']
#     attention_mask = inputs['attention_mask']
    
#     with torch.no_grad():
#         output = model(input_ids, attention_mask=attention_mask)

#     answer_start = torch.argmax(output.start_logits)
#     answer_end = torch.argmax(output.end_logits)
#     answer_start_i=int(answer_start/512)
#     answer_start_j=int(answer_start%512)

#     answer_end_j=int(answer_end%512)
        
    
#     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))    
#     return answer
from collections import Counter

def get_answers(question, context):
    inputs = context_conversion(question, context)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    all_answers = []  # To store answers from each chunk
    
    with torch.no_grad():
        for i in range(input_ids.size(0)):
            chunk_input_ids = input_ids[i]
            chunk_attention_mask = attention_mask[i]
            
            # Reshape chunk_input_ids and chunk_attention_mask to remove padding
            chunk_input_ids = chunk_input_ids[chunk_input_ids != 0]
            chunk_attention_mask = chunk_attention_mask[chunk_attention_mask != 0]
            
            output = model(chunk_input_ids.unsqueeze(0), attention_mask=chunk_attention_mask.unsqueeze(0))
            
            answer_start = torch.argmax(output.start_logits)
            answer_end = torch.argmax(output.end_logits)
            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(chunk_input_ids[answer_start:answer_end + 1]))
            all_answers.append(answer)
    
    # Count the occurrences of each answer
    answer_counts = Counter(all_answers)
    
    # Select the most frequent answer
    most_frequent_answer = answer_counts.most_common(1)[0][0]
    
    return most_frequent_answer


In [ ]:
question="what is CEFC?"
print(f"Predicted Answer: {get_answers(question,context)}")

In [ ]:
question="What is internship 2023 ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="what are services ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="who are the contributors ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="what is dewg ?"
print(f"Predicted Answer: {get_answers(question, context)}")


In [ ]:
question="what is the process for Internship 2023 ?"
print(f"Predicted Answer: {get_answers(question, context)}")


In [ ]:
question="what is the FOUNDATION FOR SMART MANUFACTURING?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="what is the FSM?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="what is the summer Internship 2023 ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
question="Expalin  Internship 2023 ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
 
question="Explain Collaborative robots ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
 
question="What Collaborative robots can do ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:

 
question="What are the applications of collaborative robots  ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
 
question="What is PLC ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
 
question="What are contact information ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:

question="Explain Digital Twin ?"
print(f"Predicted Answer: {get_answers(question, context)}")

In [ ]:
 
question="what are Digital twin business values ?"
print(f"Predicted Answer: {get_answers(question, context)}")